In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

# Load the data
data = pd.read_excel("YourExcelFile.xlsx")

# Filter data for DrawDay: Tuesday
select_data = data[data['DrawDay'] == 'Tuesday']

# Extract 'Ball 1', 'Ball 2', 'Ball 3', 'Ball 4', 'Ball 5' columns
X = select_data[['Ball 1', 'Ball 2', 'Ball 3', 'Ball 4', 'Ball 5']]

# Convert X to a NumPy array
X = X.values

# Define the model using original TensorFlow
model_original = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(5,)),
    tf.keras.layers.Dense(5)  # Output layer with 5 units
])
model_original.compile(optimizer='adam', loss='mean_squared_error')
model_original.fit(X, X, epochs=10, batch_size=32)

# Define the model using TensorFlow AutoML
input_shape = (5,)
model_automl = keras.Sequential([
    preprocessing.Normalization(input_shape=input_shape),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])
model_automl.compile(optimizer='adam', loss='mean_squared_error')
model_automl.fit(X, X, epochs=10, batch_size=32)

# Predict using the last 10 rows of the filtered data
next_draw_data = X[-10:]
next_draw_prediction_original = model_original.predict(next_draw_data)
next_draw_prediction_automl = model_automl.predict(next_draw_data)

# Display predictions using original TensorFlow
print("Next draw prediction (using TensorFlow):")
for prediction in next_draw_prediction_original:
    sorted_prediction = sorted([int(round(p)) for p in prediction])
    print(" ".join(map(str, sorted_prediction)))

# Display predictions sorted for each ball using TensorFlow AutoML
sorted_predictions_automl = next_draw_prediction_automl.argsort(axis=0)
print("Next draw predictions (sorted) using TensorFlow AutoML:")
for i, ball in enumerate(['Ball 1', 'Ball 2', 'Ball 3', 'Ball 4', 'Ball 5']):
    print(f"{ball}: {next_draw_data[:, i][sorted_predictions_automl[i]]}")
